In [8]:
# %run "wall_of_imports.ipynb"
# %run "agent_state.ipynb"

In [9]:
def accommodation_agent(state: AgentState) -> dict:
    """Provides hotel and accommodation recommendations."""
    try:
        # Extract accommodation preferences
        extraction_prompt = ChatPromptTemplate.from_messages([
            ("system", """Extract accommodation preferences from the user's query.
            Return a JSON object with these fields (leave empty if not mentioned):
            {{
                "location": "city or specific area",
                "check_in_date": "in YYYY-MM-DD format",
                "check_out_date": "in YYYY-MM-DD format",
                "guests": "number of guests",
                "rooms": "number of rooms",
                "budget_range": "price range per night"
            }}
            """),  # Note the double curly braces
            ("human", "{query}")
        ])
        
        extraction_chain = extraction_prompt | llm | StrOutputParser()
        try:
            accommodation_params = json.loads(extraction_chain.invoke({"query": state.query}))
            state.context.update({"accommodation_params": accommodation_params})
        except json.JSONDecodeError:
            state.context.update({"accommodation_params": {}})
        
        # Get accommodation information using RAG
        rag_chain = setup_rag_chain()
        retrieval_result = rag_chain.invoke({"query": f"hotels in {state.context.get('accommodation_params', {}).get('location', '')}"})
        
        # Generate response based on accommodation parameters and retrieved information
        accommodation_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a hotel and accommodation expert. Provide detailed recommendations
            based on the user's preferences and the retrieved accommodation data. Include:
            
            - Suitable hotel/accommodation options
            - Price ranges and value considerations
            - Location benefits and proximity to attractions
            - Amenities and facilities
            - Guest ratings and reviews summary
            - Booking tips and optimal timing
            
            If specific accommodation data isn't available, provide general advice about
            accommodations in the requested location, typical options at different price points,
            and best areas to stay.
            
            Retrieved accommodation information:
            {context}
            
            Extracted accommodation parameters:
            {accommodation_params}
            """),
            ("human", "{query}")
        ])
        
        accommodation_chain = accommodation_prompt | llm | StrOutputParser()
        response = accommodation_chain.invoke({
            "input": state.query,
            "context": retrieval_result.get("context", ""),
            "accommodation_params": json.dumps(state.context.get("accommodation_params", {}), indent=2)
        })
        
        return {"agent_response": response}
        
    except Exception as e:
        return {"error": str(e)}